In [ ]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 데이터 파일 로드
normal_data = pd.read_csv('C:/Users/7-2009-1186-16/Desktop/-sampling-frequency-hz-/vibration_data/re_condition1/300rpm(diff)/Normal/resampled_normal_data_2023_12_7_17_2.csv')
faulty_data = pd.read_csv('C:/Users/7-2009-1186-16/Desktop/-sampling-frequency-hz-/vibration_data/re_condition1/300rpm(diff)/Fault1/resampled_fault_data_2023_12_14_13_52.csv')

# 정상 데이터에 레이블 0(정상) 할당, 비정상 데이터에 레이블 1(비정상) 할당
normal_data['label'] = 0
faulty_data['label'] = 1

# 데이터 결합
combined_data = pd.concat([normal_data, faulty_data], ignore_index=True)

# X, Y, Z 축 데이터와 레이블 선택
features = combined_data[['motor_x', 'motor_y', 'motor_z']]
labels = combined_data['label']

# 데이터 정규화
scaler = StandardScaler()
X = scaler.fit_transform(features)

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=30)

# 1D CNN 입력을 위해 데이터 형태 변경
# (샘플 수, 시퀀스 길이, 채널 수)
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)

# CNN 모델 생성
model = Sequential()
model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'))
model.add(MaxPooling1D(pool_size=1))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # 이진 분류 출력 레이어

# 모델 컴파일
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 훈련
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

model.summury